In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime
import time

In [5]:
# Read the cleaned CSV data
data = pd.read_csv(r"C:\Users\Admin\OneDrive\Desktop\drmaths\Hospital Length of Stay Project\LengthOfStay.csv")

In [7]:
# Convert date columns to datetime and ensure they are in 'YYYY-MM-DD' format

data['vdate'] = pd.to_datetime(data['vdate'], errors='coerce').dt.strftime('%Y-%m-%d')
data['discharged'] = pd.to_datetime(data['discharged'], errors='coerce').dt.strftime('%Y-%m-%d')

In [11]:
# Replace missing values with a default or None for null insertions in MySQL
data.fillna({'secondarydiagnosisnonicd9': 'Unknown'}, inplace=True)

In [13]:
print("Data has been read and processed successfully.")

Data has been read and processed successfully.


In [21]:
# Function to establish a connection without specifying a database

def get_connection_without_db():
    attempts = 3  # Number of reconnection attempts
    while attempts > 0:
        try:
            conn = mysql.connector.connect(
                host='localhost',
                user='root',  # Replace with your MySQL username
                password='********'  # Replace with your MySQL password
            )
            print("Successfully connected to MySQL (without specifying a database)")
            return conn
        except mysql.connector.Error as err:
            print(f"Error: {err}. Retrying in 5 seconds...")
            time.sleep(5)
            attempts -= 1
    raise Exception("Could not connect to the MySQL server after several attempts.")

# Establish the connection without specifying a database
conn = get_connection_without_db()
cursor = conn.cursor()

# Create the database if it doesn't exist and then switch to it
cursor.execute("CREATE DATABASE IF NOT EXISTS HospitalLengthOfStayDW")
cursor.execute("USE HospitalLengthOfStayDW")
print("Database 'HospitalLengthOfStayDW' is ready and in use.")

# Close the initial connection
cursor.close()
conn.close()

# Now reconnect with the database specified
def get_connection():
    attempts = 3  # Number of reconnection attempts
    while attempts > 0:
        try:
            conn = mysql.connector.connect(
                host='localhost',
                user='root',  # Replace with your MySQL username
                password='TOLATs17$',  # Replace with your MySQL password
                database='HospitalLengthOfStayDW'
            )
            print("Successfully connected to MySQL with 'HospitalLengthOfStayDW'")
            return conn
        except mysql.connector.Error as err:
            print(f"Error: {err}. Retrying in 5 seconds...")
            time.sleep(5)
            attempts -= 1
    raise Exception("Could not connect to the MySQL server after several attempts.")

# Establish the connection to the specified database
conn = get_connection()
cursor = conn.cursor(buffered=True)

Successfully connected to MySQL (without specifying a database)
Database 'HospitalLengthOfStayDW' is ready and in use.
Successfully connected to MySQL with 'HospitalLengthOfStayDW'


In [23]:
# Drop existing database and create a fresh one

cursor.execute("DROP DATABASE IF EXISTS HospitalLengthOfStayDW")
cursor.execute("CREATE DATABASE HospitalLengthOfStayDW")
cursor.execute("USE HospitalLengthOfStayDW")

# Create the dimension and fact tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Dim_Date (
        Date_ID INT PRIMARY KEY AUTO_INCREMENT,
        Date DATE UNIQUE,
        Year INT,
        Month INT,
        Day INT,
        Week INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Dim_Patient (
        Patient_ID INT PRIMARY KEY AUTO_INCREMENT,
        Gender VARCHAR(10),
        Dialysis_Renal_Endstage BOOLEAN,
        Asthma BOOLEAN,
        Iron_Deficiency BOOLEAN,
        Pneumonia BOOLEAN,
        Substance_Dependence BOOLEAN,
        Psychological_Disorder BOOLEAN,
        Depression BOOLEAN,
        Psychotherapy BOOLEAN,
        Fibrosis BOOLEAN,
        Malnutrition BOOLEAN
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Dim_Diagnosis (
        Diagnosis_ID INT PRIMARY KEY AUTO_INCREMENT,
        Secondary_Diagnosis VARCHAR(255) UNIQUE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Dim_Facility (
        Facility_ID INT PRIMARY KEY AUTO_INCREMENT,
        Facility_Code VARCHAR(5) UNIQUE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Fact_Stay (
        Stay_ID INT PRIMARY KEY AUTO_INCREMENT,
        Patient_ID INT,
        Date_ID INT,
        Diagnosis_ID INT,
        Facility_ID INT,
        Hematocrit DECIMAL(5, 2),
        Neutrophils DECIMAL(5, 2),
        Sodium DECIMAL(6, 2),
        Glucose DECIMAL(6, 2),
        Blood_Urea_Nitrogen DECIMAL(5, 2),
        Creatinine DECIMAL(5, 2),
        BMI DECIMAL(5, 2),
        Pulse INT,
        Respiration DECIMAL(3, 1),
        Length_Of_Stay INT,
        FOREIGN KEY (Patient_ID) REFERENCES Dim_Patient(Patient_ID),
        FOREIGN KEY (Date_ID) REFERENCES Dim_Date(Date_ID),
        FOREIGN KEY (Diagnosis_ID) REFERENCES Dim_Diagnosis(Diagnosis_ID),
        FOREIGN KEY (Facility_ID) REFERENCES Dim_Facility(Facility_ID)
    )
''')

# Commit the table creation
conn.commit()
print("Tables created successfully.")

Tables created successfully.


In [25]:
def get_or_create_date_id(date):
    if not pd.isna(date):
        cursor.execute('SELECT Date_ID FROM Dim_Date WHERE Date = %s', (date,))
        result = cursor.fetchone()
        if result:
            return result[0]
        else:
            date_obj = datetime.strptime(date, '%Y-%m-%d')
            cursor.execute('''
                INSERT INTO Dim_Date (Date, Year, Month, Day, Week)
                VALUES (%s, %s, %s, %s, %s)
            ''', (date, date_obj.year, date_obj.month, date_obj.day, date_obj.isocalendar()[1]))
            conn.commit()
            return cursor.lastrowid
    return None

def get_or_create_patient_id(row):
    cursor.execute('''
        SELECT Patient_ID FROM Dim_Patient WHERE 
        Gender = %s AND Dialysis_Renal_Endstage = %s AND Asthma = %s AND Iron_Deficiency = %s AND
        Pneumonia = %s AND Substance_Dependence = %s AND Psychological_Disorder = %s AND
        Depression = %s AND Psychotherapy = %s AND Fibrosis = %s AND Malnutrition = %s
    ''', (
        row['gender'], row['dialysisrenalendstage'], row['asthma'], row['irondef'],
        row['pneum'], row['substancedependence'], row['psychologicaldisordermajor'],
        row['depress'], row['psychother'], row['fibrosisandother'], row['malnutrition']
    ))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute('''
            INSERT INTO Dim_Patient (
                Gender, Dialysis_Renal_Endstage, Asthma, Iron_Deficiency, Pneumonia, 
                Substance_Dependence, Psychological_Disorder, Depression, Psychotherapy, 
                Fibrosis, Malnutrition
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            row['gender'], row['dialysisrenalendstage'], row['asthma'], row['irondef'],
            row['pneum'], row['substancedependence'], row['psychologicaldisordermajor'],
            row['depress'], row['psychother'], row['fibrosisandother'], row['malnutrition']
        ))
        conn.commit()
        return cursor.lastrowid

def get_or_create_diagnosis_id(diagnosis):
    cursor.execute('SELECT Diagnosis_ID FROM Dim_Diagnosis WHERE Secondary_Diagnosis = %s', (diagnosis,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute('INSERT INTO Dim_Diagnosis (Secondary_Diagnosis) VALUES (%s)', (diagnosis,))
        conn.commit()
        return cursor.lastrowid

def get_or_create_facility_id(facility_code):
    cursor.execute('SELECT Facility_ID FROM Dim_Facility WHERE Facility_Code = %s', (facility_code,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute('INSERT INTO Dim_Facility (Facility_Code) VALUES (%s)', (facility_code,))
        conn.commit()
        return cursor.lastrowid

In [27]:
# Prepare and load data into Fact_Stay
for _, row in data.iterrows():
    patient_id = get_or_create_patient_id(row)
    vdate_id = get_or_create_date_id(row['vdate'])
    diagnosis_id = get_or_create_diagnosis_id(row['secondarydiagnosisnonicd9'])
    facility_id = get_or_create_facility_id(row['facid'])

    if patient_id and vdate_id and diagnosis_id and facility_id:
        cursor.execute('''
            INSERT INTO Fact_Stay (
                Patient_ID, Date_ID, Diagnosis_ID, Facility_ID, Hematocrit, Neutrophils, Sodium, Glucose, 
                Blood_Urea_Nitrogen, Creatinine, BMI, Pulse, Respiration, Length_Of_Stay
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', (
            patient_id, vdate_id, diagnosis_id, facility_id,
            row['hematocrit'], row['neutrophils'], row['sodium'], row['glucose'],
            row['bloodureanitro'], row['creatinine'], row['bmi'], row['pulse'],
            row['respiration'], row['lengthofstay']
        ))

# Commit the final data load to ensure all changes are saved
conn.commit()
print("Data loaded into Fact_Stay table successfully.")

Data loaded into Fact_Stay table successfully.


In [29]:
# Close the connection to the database

cursor.close()
conn.close()
print("MySQL connection closed.")

MySQL connection closed.
